In [1]:
from IPython.core.display import HTML, display
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
# Specify the path to the fastai directory
lib_PATH = '/Users/kunal/Desktop/MSAN_Coursework/621_ML/fastai/fastai'   # Update this
!cd {lib_PATH}
# Create sym-link to the fastai library
!ln -s {lib_PATH} ./

ln: .//fastai: File exists


In [4]:
# Load required modules
from fastai.imports import *
from fastai.structured import *
from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display
from sklearn import metrics
from dateutil.relativedelta import relativedelta

# Load Data

## `train.csv`

In [5]:
PATH = '../data/'

In [6]:
train = pd.read_csv(f'{PATH}train.csv')
train.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [7]:
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)
display_all(train.describe(include='all').transpose())

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
msno,7377418,30755,MXIMDXO0j3UpaT7FvOSGW6Y5zfhlh+xYjTqGoUdMzEE=,5819,NaN,NaN,NaN,NaN,NaN,NaN,NaN
song_id,7377418,359966,reXuGcEWDDCnL0K3Th//3DFG4S1ACSpJMzA+CFipo1g=,13973,NaN,NaN,NaN,NaN,NaN,NaN,NaN
source_system_tab,7352569,8,my library,3684730,NaN,NaN,NaN,NaN,NaN,NaN,NaN
source_screen_name,6962614,20,Local playlist more,3228202,NaN,NaN,NaN,NaN,NaN,NaN,NaN
source_type,7355879,12,local-library,2261399,NaN,NaN,NaN,NaN,NaN,NaN,NaN
target,7.37742e+06,NaN,NaN,NaN,0.503517,0.499988,0,0,1,1,1


In [8]:
train.isnull().any()

msno                  False
song_id               False
source_system_tab      True
source_screen_name     True
source_type            True
target                False
dtype: bool

## `songs.csv`

In [9]:
songs = pd.read_csv(f'{PATH}songs.csv')

In [10]:
songs.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [11]:
songs.isnull().any()

song_id        False
song_length    False
genre_ids       True
artist_name    False
composer        True
lyricist        True
language        True
dtype: bool

In [12]:
display_all(songs.describe(include='all').transpose())

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
song_id,2296320,2296320,04wva3sKANDiCQDQ5jqXOsypdGydSCIKh3emFId/WIE=,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
song_length,2.29632e+06,NaN,NaN,NaN,246993,160920,185,183600,226627,277269,1.21738e+07
genre_ids,2202204,1045,465,567911,NaN,NaN,NaN,NaN,NaN,NaN,NaN
artist_name,2296320,222363,Various Artists,145916,NaN,NaN,NaN,NaN,NaN,NaN,NaN
composer,1224966,329823,Neuromancer,17888,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lyricist,351052,110925,Traditional,1751,NaN,NaN,NaN,NaN,NaN,NaN,NaN
language,2.29632e+06,NaN,NaN,NaN,32.378,24.3324,-1,-1,52,52,59


## `members.csv`

In [13]:
members = pd.read_csv(f'{PATH}members.csv', parse_dates=['registration_init_time','expiration_date'])
members.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,2011-08-20,2017-09-20
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,2015-06-28,2017-06-22
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,2016-04-11,2017-07-12
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,2015-09-06,2015-09-07
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,2017-01-26,2017-06-13


In [14]:
display_all(members.describe(include='all').transpose())

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
msno,34403,34403,z81IiiZlppoV587E7OQ1+0MayYELS1wVs3ANw5H6p+E=,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
city,34403,NaN,NaN,NaN,NaN,NaN,5.37128,6.24393,1,1,1,10,22
bd,34403,NaN,NaN,NaN,NaN,NaN,12.2809,18.1703,-43,0,0,25,1051
gender,14501,2,male,7405,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
registered_via,34403,NaN,NaN,NaN,NaN,NaN,5.95338,2.28753,3,4,7,9,16
registration_init_time,34403,3862,2017-01-26 00:00:00,117,2004-03-26 00:00:00,2017-02-28 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
expiration_date,34403,1484,2017-09-30 00:00:00,2498,1970-01-01 00:00:00,2020-10-17 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## `song_extra_info.csv`

In [15]:
songs_extra = pd.read_csv(f'{PATH}song_extra_info.csv')
songs_extra.head()

,song_id,name,isrc
0,LP7pLJoJFBvyuUwvu+oLzjT+bI+UeBPURCecJsX1jjs=,我們,TWUM71200043
1,ClazTFnk6r0Bnuie44bocdNMM3rdlrq0bCGAsGUWcHE=,Let Me Love You,QMZSY1600015
2,u2ja/bZE3zhCGxvbbOB3zOoUjx27u40cf5g09UXMoKQ=,原諒我,TWA530887303
3,92Fqsy0+p6+RHe2EoLKjHahORHR1Kq1TBJoClW9v+Ts=,Classic,USSM11301446
4,0QFmz/+rJy1Q56C1DuYqT9hKKqi5TUqx0sN0IwvoHrw=,愛投羅網,TWA471306001


In [16]:
display_all(songs_extra.describe(include='all').transpose())

,count,unique,top,freq
song_id,2295971,2295971,04wva3sKANDiCQDQ5jqXOsypdGydSCIKh3emFId/WIE=,1
name,2295969,1168979,Intro,1734
isrc,2159423,1806825,GBPS81518952,207


## `test.csv`

In [17]:
test = pd.read_csv(f'{PATH}test.csv')
test.head()

,id,msno,song_id,source_system_tab,source_screen_name,source_type
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library
2,2,/uQAlrAkaczV+nWCd2sPF2ekvXPRipV7q0l+gbLuxjw=,8eZLFOdGVdXBSqoAv5nsLigeH2BvKXzTQYtUM53I0k4=,discover,NaN,song-based-playlist
3,3,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,ztCf8thYsS4YN3GcIL/bvoxLm/T5mYBVKOO4C9NiVfQ=,radio,Radio,radio
4,4,1a6oo/iXKatxQx4eS9zTVD+KlSVaAFbTIqVvwLC1Y0k=,MKVMpslKcQhMaFEgcEQhEfi5+RZhMYlU3eRDpySrH8Y=,radio,Radio,radio


In [18]:
display_all(test.describe(include='all').transpose())

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
id,2.55679e+06,NaN,NaN,NaN,1.27839e+06,738082,0,639197,1.27839e+06,1.91759e+06,2.55679e+06
msno,2556790,25131,KGXNZ/H3VxvET/+rGxlrAe7Gpz2eKMXyuSg3xh8Ij1M=,2489,NaN,NaN,NaN,NaN,NaN,NaN,NaN
song_id,2556790,224753,ZcKgNis1AP1LA0sdtIddrtk7P04iiJzJrXvwXdT/X3Q=,8320,NaN,NaN,NaN,NaN,NaN,NaN,NaN
source_system_tab,2548348,8,my library,1019492,NaN,NaN,NaN,NaN,NaN,NaN,NaN
source_screen_name,2393907,22,Local playlist more,845115,NaN,NaN,NaN,NaN,NaN,NaN,NaN
source_type,2549493,12,online-playlist,774532,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Merge Data & Feature Engineering

In [19]:
train = train.merge(songs, on='song_id', how='left')

In [20]:
test = test.merge(songs, on='song_id', how='left')

In [21]:
members['membership_days'] = members['expiration_date'].subtract(members['registration_init_time']).dt.days.astype(int)

In [22]:
members['registration_year'] = members['registration_init_time'].dt.year
members['registration_month'] = members['registration_init_time'].dt.month
members['registration_date'] = members['registration_init_time'].dt.day

In [23]:
members['expiration_year'] = members['expiration_date'].dt.year
members['expiration_month'] = members['expiration_date'].dt.month
members['expiration_date'] = members['expiration_date'].dt.day

In [24]:
members.head()

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date,membership_days,registration_year,registration_month,registration_date,expiration_year,expiration_month
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,NaN,7,2011-08-20,20,2223,2011,8,20,2017,9
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,NaN,7,2015-06-28,22,725,2015,6,28,2017,6
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,NaN,4,2016-04-11,12,457,2016,4,11,2017,7
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,NaN,9,2015-09-06,7,1,2015,9,6,2015,9
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,NaN,4,2017-01-26,13,138,2017,1,26,2017,6


In [25]:
def isrc_to_year(isrc):
    if type(isrc) == str:
        if int(isrc[5:7]) > 17:
            return 1900 + int(isrc[5:7])
        else:
            return 2000 + int(isrc[5:7])
    else:
        return np.nan

In [26]:
songs_extra['song_year'] = songs_extra['isrc'].apply(isrc_to_year)

In [27]:
train = train.merge(members, on='msno', how='left')
test = test.merge(members, on='msno', how='left')

In [28]:
train = train.merge(songs_extra, on = 'song_id', how = 'left')
train.song_length.fillna(200000,inplace=True)
train.song_length = train.song_length.astype(np.uint32)
train.song_id = train.song_id.astype('category')

In [29]:
test = test.merge(songs_extra, on = 'song_id', how = 'left')
test.song_length.fillna(200000,inplace=True)
test.song_length = test.song_length.astype(np.uint32)
test.song_id = test.song_id.astype('category')

In [30]:
train_cats(train)
train_cats(test)

In [31]:
display_all(train.isnull().sum().sort_index()/len(train))

artist_name               0.000015
bd                        0.000000
city                      0.000000
composer                  0.227140
expiration_date           0.000000
expiration_month          0.000000
expiration_year           0.000000
gender                    0.401425
genre_ids                 0.016056
isrc                      0.078328
language                  0.000020
lyricist                  0.430882
membership_days           0.000000
msno                      0.000000
name                      0.000197
registered_via            0.000000
registration_date         0.000000
registration_init_time    0.000000
registration_month        0.000000
registration_year         0.000000
song_id                   0.000000
song_length               0.000000
song_year                 0.078328
source_screen_name        0.056226
source_system_tab         0.003368
source_type               0.002920
target                    0.000000
dtype: float64

In [32]:
train.head(2)

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,composer,...,expiration_date,membership_days,registration_year,registration_month,registration_date,expiration_year,expiration_month,name,isrc,song_year
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,206471,359,Bastille,Dan Smith| Mark Crew,...,5,2103,2012,1,2,2017,10,Good Grief,GBUM71602854,2016.0
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1,284584,1259,Various Artists,NaN,...,11,2301,2011,5,25,2017,9,Lords of Cardboard,US3C69910183,1999.0


In [33]:
test.head(2)

,id,msno,song_id,source_system_tab,source_screen_name,source_type,song_length,genre_ids,artist_name,composer,...,expiration_date,membership_days,registration_year,registration_month,registration_date,expiration_year,expiration_month,name,isrc,song_year
0,0,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,WmHKgKMlp1lQMecNdNvDMkvIycZYHnFwDT72I5sIssc=,my library,Local playlist more,local-library,224130,458,梁文音 (Rachel Liang),Qi Zheng Zhang,...,18,577,2016,2,19,2017,9,愛其實很殘忍,TWUM71400047,2014.0
1,1,V8ruy7SGk7tDm3zA51DPpn6qutt+vmKMBKa21dp54uM=,y/rsZ9DC7FwK5F2PK2D5mj+aOBUJAjuu3dZ14NgE0vM=,my library,Local playlist more,local-library,320470,465,林俊傑 (JJ Lin),林俊傑,...,18,577,2016,2,19,2017,9,她說,TWB671005201,2010.0


In [34]:
train.dtypes

msno                            category
song_id                         category
source_system_tab               category
source_screen_name              category
source_type                     category
target                             int64
song_length                       uint32
genre_ids                       category
artist_name                     category
composer                        category
lyricist                        category
language                         float64
city                               int64
bd                                 int64
gender                          category
registered_via                     int64
registration_init_time    datetime64[ns]
expiration_date                    int64
membership_days                    int64
registration_year                  int64
registration_month                 int64
registration_date                  int64
expiration_year                    int64
expiration_month                   int64
name            

# More Features

In [35]:
def genre_id_count(x):
    if x == 'no_genre_id':
        return 0
    else:
        return x.count('|') + 1

train['genre_ids'] = train['genre_ids'].cat.add_categories(['no_genre_id'])    # cat accesses categorical properties
test['genre_ids'] = test['genre_ids'].cat.add_categories(['no_genre_id'])

train['genre_ids'].fillna('no_genre_id', inplace=True)
test['genre_ids'].fillna('no_genre_id', inplace=True)
train['genre_ids_count'] = train['genre_ids'].apply(genre_id_count).astype(np.int8)
test['genre_ids_count'] = test['genre_ids'].apply(genre_id_count).astype(np.int8)

In [36]:
def lyricist_count(x):
    if x == 'no_lyricist':
        return 0
    else:
        return sum(map(x.count, ['|', '/', '\\', ';'])) + 1
    return sum(map(x.count, ['|', '/', '\\', ';']))

train['lyricist'] = train['lyricist'].cat.add_categories(['no_lyricist'])
train['lyricist'].fillna('no_lyricist', inplace=True)
train['lyricists_count'] = train['lyricist'].apply(lyricist_count).astype(np.int8)
test['lyricist'] = test['lyricist'].cat.add_categories(['no_lyricist'])
test['lyricist'].fillna('no_lyricist', inplace=True)
test['lyricists_count'] = test['lyricist'].apply(lyricist_count).astype(np.int8)

In [37]:
def composer_count(x):
    if x == 'no_composer':
        return 0
    else:
        return sum(map(x.count, ['|', '/', '\\', ';'])) + 1

train['composer'] = train['composer'].cat.add_categories(['no_composer'])
train['composer'].fillna('no_composer',inplace=True)
train['composer_count'] = train['composer'].apply(composer_count).astype(np.int8)
test['composer'] = test['composer'].cat.add_categories(['no_composer'])
test['composer'].fillna('no_composer',inplace=True)
test['composer_count'] = test['composer'].apply(composer_count).astype(np.int8)

In [38]:
def is_featured(x):
    if 'feat' in str(x) :
        return 1
    return 0

train['artist_name'] = train['artist_name'].cat.add_categories(['no_artist'])
train['artist_name'].fillna('no_artist',inplace=True)
train['is_featured'] = train['artist_name'].apply(is_featured).astype(np.int8)
test['artist_name'] = test['artist_name'].cat.add_categories(['no_artist'])
test['artist_name'].fillna('no_artist',inplace=True)
test['is_featured'] = test['artist_name'].apply(is_featured).astype(np.int8)

In [39]:
def artist_count(x):
    if x == 'no_artist':
        return 0
    else:
        return x.count('and') + x.count(',') + x.count('feat') + x.count('&')

train['artist_count'] = train['artist_name'].apply(artist_count).astype(np.int8)
test['artist_count'] = test['artist_name'].apply(artist_count).astype(np.int8)

# if artist is same as composer
train['artist_composer'] = (np.asarray(train['artist_name']) == np.asarray(train['composer'])).astype(np.int8)
test['artist_composer'] = (np.asarray(test['artist_name']) == np.asarray(test['composer'])).astype(np.int8)

In [40]:
# if artist, lyricist and composer are all three same
train['artist_composer_lyricist'] = ((np.asarray(train['artist_name']) == np.asarray(train['composer'])) & 
                                     np.asarray((train['artist_name']) == np.asarray(train['lyricist'])) & 
                                     np.asarray((train['composer']) == np.asarray(train['lyricist']))).astype(np.int8)
test['artist_composer_lyricist'] = ((np.asarray(test['artist_name']) == np.asarray(test['composer'])) & 
                                    (np.asarray(test['artist_name']) == np.asarray(test['lyricist'])) &
                                    np.asarray((test['composer']) == np.asarray(test['lyricist']))).astype(np.int8)

In [41]:
# is song language 17 or 45. 
def song_lang_boolean(x):
    if '17.0' in str(x) or '45.0' in str(x):
        return 1
    return 0

train['song_lang_boolean'] = train['language'].apply(song_lang_boolean).astype(np.int8)
test['song_lang_boolean'] = test['language'].apply(song_lang_boolean).astype(np.int8)

In [42]:
_median_song_length = np.median(train['song_length'])
def smaller_song(x):
    if x < _median_song_length:
        return 1
    return 0

train['smaller_song'] = train['song_length'].apply(smaller_song).astype(np.int8)
test['smaller_song'] = test['song_length'].apply(smaller_song).astype(np.int8)

In [43]:
# number of times a song has been played before
_dict_count_song_played_train = {k: v for k, v in train['song_id'].value_counts().iteritems()}
_dict_count_song_played_test = {k: v for k, v in test['song_id'].value_counts().iteritems()}
def count_song_played(x):
    try:
        return _dict_count_song_played_train[x]
    except KeyError:
        try:
            return _dict_count_song_played_test[x]
        except KeyError:
            return 0
        
train['count_song_played'] = train['song_id'].apply(count_song_played).astype(np.int64)
test['count_song_played'] = test['song_id'].apply(count_song_played).astype(np.int64)

In [44]:
# number of times the artist has been played
_dict_count_artist_played_train = {k: v for k, v in train['artist_name'].value_counts().iteritems()}
_dict_count_artist_played_test = {k: v for k, v in test['artist_name'].value_counts().iteritems()}
def count_artist_played(x):
    try:
        return _dict_count_artist_played_train[x]
    except KeyError:
        try:
            return _dict_count_artist_played_test[x]
        except KeyError:
            return 0
        
train['count_artist_played'] = train['artist_name'].apply(count_artist_played).astype(np.int64)
test['count_artist_played'] = test['artist_name'].apply(count_artist_played).astype(np.int64)

In [45]:
# train['registration_year'][train['registration_year']==np.inf] = np.nan
train.song_year.replace([np.inf, -np.inf], np.nan, inplace=True)

In [46]:
train.song_year.fillna(-1,inplace=True)

In [47]:
train.song_year = train.song_year.astype('int64')

In [48]:
train = train.drop(['registration_init_time'], axis=1)

In [49]:
test = test.drop(['registration_init_time'], axis=1)

In [50]:
train = train.drop(['isrc'], axis=1)
test = test.drop(['isrc'], axis=1)

In [51]:
train.language.fillna(-1,inplace=True)
train.language = train.language.astype('int8')

In [52]:
test.language.fillna(-1,inplace=True)
test.language = test.language.astype('int8')

In [53]:
# train['source_system_tab'] = train['source_system_tab'].cat.add_categories(['unk_source_system_tab'])
# train['source_system_tab'].fillna('unk_source_system_tab',inplace=True)
# test['source_system_tab'] = test['source_system_tab'].cat.add_categories(['unk_source_system_tab'])
# test['source_system_tab'].fillna('unk_source_system_tab',inplace=True)

# train['source_screen_name'] = train['source_screen_name'].cat.add_categories(['unk_source_screen_name'])
# train['source_screen_name'].fillna('unk_source_screen_name',inplace=True)
# test['source_screen_name'] = test['source_screen_name'].cat.add_categories(['unk_source_screen_name'])
# test['source_screen_name'].fillna('unk_source_screen_name',inplace=True)

# train['source_type'] = train['source_type'].cat.add_categories(['unk_source_type'])
# train['source_type'].fillna('unk_source_type',inplace=True)
# test['source_type'] = test['source_type'].cat.add_categories(['unk_source_type'])
# test['source_type'].fillna('unk_source_type',inplace=True)

# train['gender'] = train['gender'].cat.add_categories(['unk_gender'])
# train['gender'].fillna('unk_gender',inplace=True)
# test['gender'] = test['gender'].cat.add_categories(['unk_gender'])
# test['gender'].fillna('unk_gender',inplace=True)

# train['name'] = train['name'].cat.add_categories(['unk_name'])
# train['name'].fillna('unk_name',inplace=True)
# test['name'] = test['name'].cat.add_categories(['unk_name'])
# test['name'].fillna('unk_name',inplace=True)

In [54]:
train.to_feather('../data/tmp/wsdm_train')
test.to_feather('../data/tmp/wsdm_test')

In [55]:
# ??proc_df

In [56]:
# ??parallel_trees